In [ ]:
#checking GPU availability
!nvidia-smi

Fri Sep 19 18:36:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#installing required libraries
!pip install --upgrade pip
!pip install --quiet datasets transformers accelerate peft bitsandbytes safetensors huggingface_hub sentencepiece scikit-learn
!apt-get install -y git-lfs > /dev/null 2>&1 || true

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('QWEN_HF_TOKEN')

In [ ]:
from huggingface_hub import login
login(token=HF_TOKEN)

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
HF_DATASET_ID = "karan842/ipc-sections"
OUTPUT_DIR = "/content/qwen_ipc_finetuned_1p5B"
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset(HF_DATASET_ID)
print(ds)
df = ds['train'].to_pandas()
df.head(3)

README.md:   0%|          | 0.00/388 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/136k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/444 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Description', 'Offense', 'Punishment', 'Section'],
        num_rows: 444
    })
})


,Description,Offense,Punishment,Section
0,Description of IPC Section 140 According to se...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,IPC_140
1,Description of IPC Section 127 According to se...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,IPC_127
2,Description of IPC Section 128 According to se...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,IPC_128


In [ ]:
import re, json
from sklearn.model_selection import train_test_split

def clean_text(s):
  if s is None: return ""
  s = str(s)
  s = re.sub(r"[\x00-\x1f\x7f]", " ", s)
  s = re.sub(r"\s+", " ", s).strip()
  return s

In [ ]:
possible_section_cols = [c for c in df.columns if 'Section' in c or 'section' in c.lower()]
possible_description_cols = [c for c in df.columns if 'Description' in c or 'description' in c.lower() or 'text' in c.lower()]
sec_col = possible_section_cols[0] if possible_section_cols else df.columns[0]
desc_col = possible_description_cols[0] if possible_description_cols else (df.columns[1] if df.shape[1]>1 else df.columns[0])

# create cleaned dataframe
df['section'] = df[sec_col].apply(clean_text)
df['description'] = df[desc_col].apply(clean_text)
# drop rows without useful description
df = df[df['description']!=''].reset_index(drop=True)
print('Rows after cleaning:', len(df))

Rows after cleaning: 444


In [ ]:
# Create instruction-style items suitable for instruct model
items = []
for _, r in df.iterrows():
  section = r['section'] if r['section'] else 'Unknown'
  desc = r['description']
  instr1 = f"Describe IPC {section} and list typical punishments. Answer succinctly."
  instr2 = f"Explain IPC {section} in simple language for a layperson."
  for instr in (instr1, instr2):
    prompt_text = "### Instruction:\n" + instr + "\n\n### Response:\n"
    completion_text = " " + desc.strip() + " "
    items.append({"prompt": prompt_text, "completion": completion_text})

In [ ]:
seen = set()
unique = []
for it in items:
  key = (it['prompt'], it['completion'])
  if key in seen: continue
  seen.add(key)
  unique.append(it)
print('Unique examples:', len(unique))

Unique examples: 884


In [ ]:
# train/test split and save JSONL
train_items, test_items = train_test_split(unique, test_size=0.1, random_state=42)


def write_jsonl(path, items):
  with open(path,'w',encoding='utf-8') as f:
    for it in items:
      f.write(json.dumps(it, ensure_ascii=False) + '\n')


train_path = os.path.join(OUTPUT_DIR, 'train.jsonl')
test_path = os.path.join(OUTPUT_DIR, 'test.jsonl')
write_jsonl(train_path, train_items)
write_jsonl(test_path, test_items)
print('Saved', len(train_items), 'train and', len(test_items), 'test to', OUTPUT_DIR)

Saved 795 train and 89 test to /content/qwen_ipc_finetuned_1p5B


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
import torch, gc, os

# free memory defensively
gc.collect()
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_8bit=True,          # keep for memory efficiency
    torch_dtype=torch.float16,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [21]:
model = prepare_model_for_kbit_training(model)

# memory savings for training
model.gradient_checkpointing_enable()
model.config.use_cache = False

In [22]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],  # adjust if your model uses different module names
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

In [23]:
# verify at least some params are trainable
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.4f}%)")

# quick check: list a few trainable parameter names
cnt = 0
for n,p in model.named_parameters():
    if p.requires_grad:
        print(n, p.shape)
        cnt += 1
        if cnt>20: break

Trainable params: 2,179,072 / 1,545,893,376 (0.1410%)
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.Size([8, 1536])
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.Size([1536, 8])
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.Size([8, 1536])
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.Size([256, 8])
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.Size([8, 1536])
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.Size([256, 8])
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight torch.Size([8, 1536])
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight torch.Size([1536, 8])
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight torch.Size([8, 1536])
base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight torch.Size([1536, 8])
base_model.model.m

In [24]:
from datasets import load_dataset
train_ds = load_dataset('json', data_files=train_path)['train']
eval_ds = load_dataset('json', data_files=test_path)['train']

max_length = 512

def tokenize_fn(example):
    text = example["prompt"] + example["completion"]
    tok = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )
    tok["labels"] = tok["input_ids"].copy()
    return tok

train_tok = train_ds.map(tokenize_fn, remove_columns=train_ds.column_names)
eval_tok  = eval_ds.map(tokenize_fn, remove_columns=eval_ds.column_names)

In [25]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding


training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=50,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    remove_unused_columns=False,
    report_to='none'
    )


data_collator = DataCollatorWithPadding(tokenizer, return_tensors='pt')


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=eval_tok,
    data_collator=data_collator
    )

In [26]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,2.081100,0.727628
2,0.620000,0.603242
3,0.544100,0.569147


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=150, training_loss=1.0817364756266277, metrics={'train_runtime': 2273.2275, 'train_samples_per_second': 1.049, 'train_steps_per_second': 0.066, 'total_flos': 9616464189849600.0, 'train_loss': 1.0817364756266277, 'epoch': 3.0})

In [27]:
# Save adapter and tokenizer
model.save_pretrained(os.path.join(OUTPUT_DIR, 'peft_adapter'))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, 'tokenizer'))
print('Saved adapter and tokenizer to', OUTPUT_DIR)

Saved adapter and tokenizer to /content/qwen_ipc_finetuned_1p5B


In [28]:
from transformers import GenerationConfig
model.eval()


num_samples = min(10, len(eval_ds))
for i in range(num_samples):
  ex = eval_ds[i]
  prompt = ex['prompt']
  inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
  gen = model.generate(**inputs, max_new_tokens=256, do_sample=False)
  out = tokenizer.decode(gen[0], skip_special_tokens=True)
  print('\nPROMPT:\n', prompt)
  print('\nMODEL OUTPUT:\n', out[len(prompt):].strip())
  print('\nREFERENCE:\n', ex['completion'].strip())
  print('\n', '-'*80)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



PROMPT:
 ### Instruction:
Describe IPC IPC_153AA and list typical punishments. Answer succinctly.

### Response:


MODEL OUTPUT:
 Description of IPC 153A: Section 153A of the Indian Penal Code (IPC) deals with the offence of "criminal intimidation". It states that whoever, by words or signs, or otherwise, puts any person in fear of death or imprisonment for life, or causes such a person to be so put in fear, shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both.

REFERENCE:
 Description of IPC Section 153AA According to section 153AA of Indian penal code, Whoever by words, either spoken or written, or by signs or by visible representations or otherwise, promotes or attempts to promote, on grounds of religion, race, place of birth, residence, language, caste or community or any other ground whatsoever, disharmony or feelings of enmity, hatred or ill-will between different religious, racials, language or regional

In [29]:
import shutil
from google.colab import files

# Define a folder for saving
SAVE_DIR = "/content/qwen2.5-finetuned"

# Save model + tokenizer
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# Zip the folder
shutil.make_archive("/content/qwen2.5-finetuned", 'zip', SAVE_DIR)

# Download zip to your computer
files.download("/content/qwen2.5-finetuned.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>